## Airline Review Extraction for Text Classification and Topic Modeling

*Install relevant packages and libraries*

In [31]:
# Install playwright library

# !pip3 install playwright   # for package

# !pip3 install asyncio      # for async calls

# !playwright install        # for browser binaries

*Import relevant packages and libraries*

In [32]:
# Import relevant libaries

import os 
import datetime 
import re

import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter 
import random

# Additional libraries

import shutil
import pprint
import numpy as np
import pandas as pd
import json
from json import loads, dumps
import string
import asyncio
from playwright.async_api import async_playwright
from playwright.sync_api import sync_playwright
from pprint import pprint 
import string  


import seaborn as sns
import plotly.express as px

from collections import Counter, defaultdict
from string import punctuation
from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix

import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from random import randint

import warnings
warnings.filterwarnings("ignore")



[nltk_data] Downloading package wordnet to /Users/dunya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Initial TripAdvisor Scraper

The goal of this project is to perform text classification and topic modeling on extracted airline reviews. In order to acquire this data, the Playwright library is utilized to scrape reviews from TripAdvisor, a popular travel website. Upon initial examination of the TripAdvisor website, it became evident that many airlines were either (1) too small to be considered or (2) not longer operating. The initial web scraper was written to extract all airlines reviewed on TripAdvisor, along with the total reviews written for each one. The methods for this are below:

*TripAdvisor Scrape Methods*

In [33]:
# Helper to concat list of lists

def flatten_sum(matrix):
    return sum(matrix, [])

# Helper to remove punctuation and clean reviews string

def conv_int(text):
    arr = text.split(' ')
    strg = arr[0].translate(str.maketrans('', '', string.punctuation))
    return strg

# Function to scrape items on TripAdvisor page

async def scrape_page(page):
    info = list()
    airline_href = page.locator('//div[@class="airlineSummary"]/a[1]')  # get HTML elements
    airline_name = page.locator('//div[@class="airlineSummary"]')
    airline_review_count = page.locator('//div[@class="airlineSummary"]/a[@class="detailsLink"]')
    
    hrefs = await airline_href.all()                        # get <a> tags
    names = await airline_name.all_inner_texts()            # get airline names
    reviews = await airline_review_count.all_inner_texts()  # get airline review counts

    idx = 0
    for idx in range(len(names)):
        name = names[idx].split('\n')[0]                    # extract airline name
        href = await hrefs[idx].get_attribute('href')       # extract href from link
        href = f"https://www.tripadvisor.com{href}"         # build out complete link
        el = {
            "airline_link": f"{href}",
            "airline_name": f"{name}",
            "airline_reviews": f"{reviews[idx]}"
        }
        info.append(el)                                     # put info into list to return
        idx += 1
    return info

# Function to go to next page

async def paginate(page, browser):
    try:
        next_page = page.locator('//span[@class="nav next ui_button primary"]')
        await next_page.click()
        return page
    except:
        await browser.close()                               # 

# Main execution function using methods defined above

async def main():
    async with async_playwright() as p:

        # Init browser
        browser = await p.chromium.launch(headless=False, slow_mo=1500)
        page = await browser.new_page()

        # Go to trip advisor
        await page.goto("https://www.tripadvisor.com/Airlines")

        # Scrape page and paginate
        airline_details = list()
        pg = 0
        while pg < 60:
            scraped = await scrape_page(page)
            airline_details.append(scraped)
            pg += 1
            await paginate(page, browser)

        # Flatten list of lists stored in airline_details
        
        # print("Length before concat: ", len(airline_details)) 
        airline_deets = flatten_sum(airline_details)
        # print("Length after concat: ", len(airline_deets))      
        return airline_deets

The use-case for each method above is as follows:

- flatten-sum: This is a helper function that takes in a list of lists and flattens, returning a one-dimensional list

- conv_int: This is a helper function that takes the total reviews string from the website, removes punctuation, and converts the value into an integer

- scrape_page: This function deals with the main extraction of text from the HTML elements contained on a review page. It identifies where the airline names and review totals are, and stores them along with links that contain the airline reviews

- paginate: This function is a scraping helper that tells the parser to move on to the next page of reviews

- main: This is the main execution function, employing all of the previous methods to gather relevant information

*Initial TripAdvisor Dataframe*

In [34]:
# Execute web scraper to get review information

information = await main()

In [ ]:
# Extract number of reviews per airline
 
airlines_df = pd.DataFrame(information)
airlines_df['review_count'] = airlines_df['airline_reviews'].apply(conv_int)
airlines_df['review_count'] = pd.to_numeric(airlines_df['review_count'])

# Get top airline reviews

airlines_df = airlines_df.sort_values('review_count', ascending=False)

The code above converts the extracted information from TripAdvisor to a pandas DataFrame. It applies the conv_int() function previously mentioned to transform the total reviews into integers. Once the review counts are numeric, we are able to sort the values in descending order.

*Save JSON*

In [ ]:
# Save scraped information
 
# Serializing json
json_object = json.dumps(information, indent=4)
 
# Writing to json file
with open("review_counts.json", "w") as outfile:
    outfile.write(json_object)

display(airlines_df.head(10))

,airline_link,airline_name,airline_reviews,review_count
557,https://www.tripadvisor.com/Airline_Review-d87...,Transavia,"109,485 reviews",109485.0
454,https://www.tripadvisor.com/Airline_Review-d87...,Ryanair,"87,393 reviews",87393.0
107,https://www.tripadvisor.com/Airline_Review-d87...,American Airlines,"77,625 reviews",77625.0
225,https://www.tripadvisor.com/Airline_Review-d87...,Emirates,"64,134 reviews",64134.0
209,https://www.tripadvisor.com/Airline_Review-d87...,Delta Air Lines,"61,679 reviews",61679.0
219,https://www.tripadvisor.com/Airline_Review-d87...,easyJet,"61,614 reviews",61614.0
160,https://www.tripadvisor.com/Airline_Review-d87...,British Airways,"58,586 reviews",58586.0
575,https://www.tripadvisor.com/Airline_Review-d87...,United Airlines,"54,120 reviews",54120.0
505,https://www.tripadvisor.com/Airline_Review-d87...,Southwest Airlines,"45,339 reviews",45339.0
350,https://www.tripadvisor.com/Airline_Review-d10...,LATAM Airlines,"40,963 reviews",40963.0


The dataframe is saved as a json file. From the table above, we are able to gauge the most highly reviewed airlines. At first, the reason for gathering this information stemmed from the idea of choosing airlines with a lot of reviews for future text classification and topic modeling. Unfortunately, after examining the most reviewed airlines on TripAdvisor, it was evident that the majority of them were low-budget. This could imply that the reviews among these airlines may be too similar.

### Skytrax Scraper

*Skytrax scraper for region information*

In [ ]:
async def skytrax():
    async with async_playwright() as p:

        # Init browser
        browser = await p.chromium.launch(headless=False, slow_mo=1500)
        page = await browser.new_page()
        region_list = list()

        # Define main url and query parameters
        og_url = "https://skytraxratings.com/airlines?regions="
        url_options = [
            "africa",
            "asia",
            "australiapacific",
            "central-america-caribbean",
            "china",
            "europe",
            "middle-east",
            "north-america",
            "russia-and-cis",
            "south-america"
        ]

        # Iterate through Skytrax pages filtered by region
        for reg in url_options:
            await page.goto(f"{og_url}{reg}")

            # Click on drop down for 50 options
            await page.select_option("select#set_posts_per_page", value="50")
            names = page.locator('//div[contains(@class,"pageportal__title")]/h2')
            titles = await names.all_inner_texts()
            region_list.append({
                f"{reg}": titles
            })
        return region_list

In order to obtain airlines less similar to one another, the information on Skytrax was leveraged. The website contained filters that allowed users to choose airlines by region. There were 10 separate regions available as options, these regions will serve as the class labels for the text classification to come. The first 50 airlines of each region were stored.

In [ ]:
region_airlines = await skytrax()

pprint(region_airlines)

[{'africa': ['Air Algerie',
             'Air Botswana',
             'Air Madagascar',
             'Air Mauritius',
             'Air Namibia',
             'Air Seychelles',
             'Arik Air',
             'Cabo Verde Airlines',
             'Egyptair',
             'Ethiopian Airlines',
             'fastjet',
             'FlySafair',
             'Kenya Airways',
             'Kulula',
             'LAM Mozambique Airlines',
             'Mango',
             'Nile Air',
             'Nouvelair',
             'Royal Air Maroc',
             'Rwandair',
             'South African Airways',
             'Tunisair']},
 {'asia': ['9 Air',
           'Air Busan',
           'Air China',
           'Air India',
           'Air India Express',
           'Air Macau',
           'AirAsia',
           'AirAsia India',
           'AirAsia X',
           'airblue',
           'ANA All Nippon Airways',
           'Asiana Airlines',
           'Bamboo Airways',
           'Bangkok Airw

In [ ]:
# Choose region specific airlines

airlines_for_reviews = [
    "South African Airways",
    "Cathay Pacific",
    "Virgin Australia",
    "Copa Airlines",
    "Air China",
    "Lufthansa",
    "Emirates",
    "Air Canada",
    "Aeroflot",
    "LATAM Airlines"
]

# Create dataframe consisting of chosen airlines

airlines_chosen = airlines_df[airlines_df["airline_name"].isin(airlines_for_reviews)]

# Helper function for assigning region to airline (based on Skytrax)

def add_region(name):
    match name:
        case "South African Airways":
            return "Africa"
        case "Cathay Pacific":
            return "Asia"
        case "Virgin Australia":
            return "Pacific"
        case "Copa Airlines":
            return "Central America"
        case "Air China":
            return "China"
        case "Lufthansa":
            return "Europe"
        case "Emirates":
            return "Middle East"
        case "Air Canada":
            return "North America"
        case "Aeroflot":
            return "Indian Subcontinent"
        case "LATAM Airlines":
            return "South America"

# Add to dataframe

airlines_chosen["region"] = airlines_chosen["airline_name"].apply(add_region)

airlines_chosen

/var/folders/t6/cjn4y6mx6lj46_h4_hhsk7740000gn/T/ipykernel_98051/2227609751.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airlines_chosen["region"] = airlines_chosen["airline_name"].apply(add_region)


,airline_link,airline_name,airline_reviews,review_count,region
225,https://www.tripadvisor.com/Airline_Review-d87...,Emirates,"64,134 reviews",64134.0,Middle East
350,https://www.tripadvisor.com/Airline_Review-d10...,LATAM Airlines,"40,963 reviews",40963.0,South America
364,https://www.tripadvisor.com/Airline_Review-d87...,Lufthansa,"40,715 reviews",40715.0,Europe
28,https://www.tripadvisor.com/Airline_Review-d87...,Air Canada,"29,753 reviews",29753.0,North America
5,https://www.tripadvisor.com/Airline_Review-d87...,Aeroflot,"20,691 reviews",20691.0,Indian Subcontinent
175,https://www.tripadvisor.com/Airline_Review-d87...,Cathay Pacific,"16,642 reviews",16642.0,Asia
198,https://www.tripadvisor.com/Airline_Review-d87...,Copa Airlines,"13,288 reviews",13288.0,Central America
591,https://www.tripadvisor.com/Airline_Review-d87...,Virgin Australia,"11,957 reviews",11957.0,Pacific
34,https://www.tripadvisor.com/Airline_Review-d87...,Air China,"6,004 reviews",6004.0,China
503,https://www.tripadvisor.com/Airline_Review-d87...,South African Airways,"4,883 reviews",4883.0,Africa


One airline from each of the ten regions was selected and a separate dataframe containing specific airlines was created. The region was added to the dataframe of the chosen airlines. The links for the airline reviews were put into a list for further web scraping below.

*Create search query list*

In [ ]:
# Get list of links for chosen airlines

search_queries = airlines_chosen['airline_link'].to_list()

print(search_queries)
print(type(search_queries))

['https://www.tripadvisor.com/Airline_Review-d8729069-Reviews-Emirates', 'https://www.tripadvisor.com/Airline_Review-d10290698-Reviews-LATAM-Airlines', 'https://www.tripadvisor.com/Airline_Review-d8729113-Reviews-Lufthansa', 'https://www.tripadvisor.com/Airline_Review-d8728998-Reviews-Air-Canada', 'https://www.tripadvisor.com/Airline_Review-d8728987-Reviews-Aeroflot', 'https://www.tripadvisor.com/Airline_Review-d8729046-Reviews-Cathay-Pacific', 'https://www.tripadvisor.com/Airline_Review-d8729055-Reviews-Copa-Airlines', 'https://www.tripadvisor.com/Airline_Review-d8728931-Reviews-Virgin-Australia', 'https://www.tripadvisor.com/Airline_Review-d8729000-Reviews-Air-China', 'https://www.tripadvisor.com/Airline_Review-d8729155-Reviews-South-African-Airways']
<class 'list'>


### Final TripAdvisor Scraper

*Scrape TripAdvisor Reviews based on extracted region information*

In [ ]:
# Function to go to next page

async def paginate_reviews(page, browser):
    try:
        next_page = page.locator('//a[@class="ui_button nav next primary "]')
        await next_page.click()
        # print('Clicked next page!!!')
        return page
    except:
        print('No next page!!!')

# Helper function to click Read more of review - Not currently used

async def click_read_more(page):
    read_more = page.locator('//div[@data-test-target="expand-review"]/span[1]')
    await read_more.all()
    return page

# Function to scrape reviews on page

async def review_scrape(page, browser):
    # Create lists to store scraped info
    reviews = list()
    idx = 0
    revc_title=[]
    revc_text=[]
    while idx < 20:                                 # 20 pages of reviews 
        page_info = list()
        await page.mouse.wheel(0, 15000)            # scroll down function
        title = page.locator('//div[@data-test-target="review-title"]/a/span/span')
        text = page.locator('//div[@data-test-target="review-title"]/following-sibling::div')
        rating = page.locator('//span[@class="ammfn"]')
        
        titles = await title.all_inner_texts()      # grab review titles
        texts = await text.all_inner_texts()        # grab review texts
        rating = await rating.all_inner_texts()     # grab airline rating

        # Merge two lists into dictionary with title and text keys
        for jdx in range(len(titles)):
            rev_title = titles[jdx]
            revc_title.append(rev_title)

            rev_raw_text = texts[jdx]
            rev_text = rev_raw_text.split("Read more")  # Split review by Read more button
            rev_text = rev_text[0]
            revc_text.append(rev_text)
            el = {                                      # Add information to relevant list
                "review_title": f"{rev_title}",
                "review_text": f"{rev_text}"
            }
            page_info.append(el)
        reviews.append(page_info)

        # Paginate
        idx += 1
        await paginate_reviews(page, browser)           # Go to next page of reviews
    return reviews, revc_title, revc_text, rating

The use-case for each method above is as follows:

- click_read_more: This is a helper function that is meant to expand the review text in the "Read more" section of a review. It is currently not being used in the project

- paginate-reviews: This function is a scraping helper that tells the parser to move on to the next page of reviews. It is used in the review_scrape() method

- review_scrape: This function deals with the main extraction of text contained in HTML elements of review pages. It identifies where the review titles and texts are on a given page, along with the airline ratings. 

Below, the functions are utilized to scrape the airline reviews:

In [ ]:
# Function to scrape reviews on page using functions above

async with async_playwright() as p:

    # Init browser

    browser = await p.chromium.launch(headless=False, slow_mo=1500)
    page = await browser.new_page()
    rev_list = list()

    # Go to trip advisor

    await page.goto("https://www.tripadvisor.com/Airlines")

    # Scrape page and paginate
    rtlist=[]
    rtextlist=[]
    airlineratinglist=[]
    for q in search_queries:
        # Go to url with reviews
        await page.goto(f"{q}")

        # Scrape relevant information
        reviews,rt,rtext,airlinerating = await review_scrape(page, browser)
        
        # Add to relevant list
        rtlist.append(rt)
        rtextlist.append(rtext)
        airlineratinglist.append(airlinerating)
        rev_list.append({
            "Airline": f"{q}",
            "Reviews": f"{reviews}"
        })

Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next page!!!
Clicked next 

In [ ]:
# Append relevant lists from code block above to dataframe

airlines_df = airlines_chosen
airlines_df['Rating']=airlineratinglist
airlines_df['Review_Title']=rtlist
airlines_df['Review_Text']=rtextlist

airlines_df

,airline_link,airline_name,airline_reviews,review_count,region,Rating,Review_Title,Review_Text
225,https://www.tripadvisor.com/Airline_Review-d87...,Emirates,"64,134 reviews",64134.0,Middle East,[4.0],"[Top customer services!!, Horrible, well below...","[Alma was so smiley, friendly and came with th..."
350,https://www.tripadvisor.com/Airline_Review-d10...,LATAM Airlines,"40,963 reviews",40963.0,South America,[3.5],"[A customer care that mirrors that 80’s, Two d...",[I have tried to change the flight 3 time. Ini...
364,https://www.tripadvisor.com/Airline_Review-d87...,Lufthansa,"40,715 reviews",40715.0,Europe,[3.5],[Bad customer service and flight cancellation ...,[Flight was cancelled and pushed back 2 days. ...
28,https://www.tripadvisor.com/Airline_Review-d87...,Air Canada,"29,753 reviews",29753.0,North America,[3.0],"[AWFUL, Worst flight experiences ever. Stay aw...","[Signed in online, paid $50 for an additional ..."
5,https://www.tripadvisor.com/Airline_Review-d87...,Aeroflot,"20,691 reviews",20691.0,Indian Subcontinent,[4.0],"[Day 320: Repayment still did not work out, WO...","[To my great surprise, I received the free, wo..."
175,https://www.tripadvisor.com/Airline_Review-d87...,Cathay Pacific,"16,642 reviews",16642.0,Asia,[4.0],"[Shocking experience...never again!, Excellent...",[Absolutely shocking customer service...meals ...
198,https://www.tripadvisor.com/Airline_Review-d87...,Copa Airlines,"13,288 reviews",13288.0,Central America,[3.5],"[Worst airline ever, Disappointed about my las...",[worst experience ever. I arrived to my destin...
591,https://www.tripadvisor.com/Airline_Review-d87...,Virgin Australia,"11,957 reviews",11957.0,Pacific,[3.5],"[Kind and knowledgeable staff., Awful customer...",[We had a terrible start to our 7.15am flight ...
34,https://www.tripadvisor.com/Airline_Review-d87...,Air China,"6,004 reviews",6004.0,China,[3.0],"[Great trip, decent food, great cabin crew, ME...",[The cabin crew was very sweet and caring. I f...
503,https://www.tripadvisor.com/Airline_Review-d87...,South African Airways,"4,883 reviews",4883.0,Africa,[3.5],"[Happy Customer’s, Avoid at all costs - zero c...",[I started writing this while in the plane. Th...


*Save Reviews as JSON*

In [ ]:
# Save Reviews to file

# Reset index
airlines_df=airlines_df.reset_index().drop(columns=['index'])

# Convert to json
reviews_json = airlines_df.to_dict("records")

# Serializing json
reviews_json = json.dumps(reviews_json, indent=4)
 
# Writing to file
with open("airline_reviews.json", "w") as outfile:
    outfile.write(reviews_json)

airlines_df

,airline_link,airline_name,airline_reviews,review_count,region,Rating,Review_Title,Review_Text
0,https://www.tripadvisor.com/Airline_Review-d87...,Emirates,"64,134 reviews",64134.0,Middle East,[4.0],"[Top customer services!!, Horrible, well below...","[Alma was so smiley, friendly and came with th..."
1,https://www.tripadvisor.com/Airline_Review-d10...,LATAM Airlines,"40,963 reviews",40963.0,South America,[3.5],"[A customer care that mirrors that 80’s, Two d...",[I have tried to change the flight 3 time. Ini...
2,https://www.tripadvisor.com/Airline_Review-d87...,Lufthansa,"40,715 reviews",40715.0,Europe,[3.5],[Bad customer service and flight cancellation ...,[Flight was cancelled and pushed back 2 days. ...
3,https://www.tripadvisor.com/Airline_Review-d87...,Air Canada,"29,753 reviews",29753.0,North America,[3.0],"[AWFUL, Worst flight experiences ever. Stay aw...","[Signed in online, paid $50 for an additional ..."
4,https://www.tripadvisor.com/Airline_Review-d87...,Aeroflot,"20,691 reviews",20691.0,Indian Subcontinent,[4.0],"[Day 320: Repayment still did not work out, WO...","[To my great surprise, I received the free, wo..."
5,https://www.tripadvisor.com/Airline_Review-d87...,Cathay Pacific,"16,642 reviews",16642.0,Asia,[4.0],"[Shocking experience...never again!, Excellent...",[Absolutely shocking customer service...meals ...
6,https://www.tripadvisor.com/Airline_Review-d87...,Copa Airlines,"13,288 reviews",13288.0,Central America,[3.5],"[Worst airline ever, Disappointed about my las...",[worst experience ever. I arrived to my destin...
7,https://www.tripadvisor.com/Airline_Review-d87...,Virgin Australia,"11,957 reviews",11957.0,Pacific,[3.5],"[Kind and knowledgeable staff., Awful customer...",[We had a terrible start to our 7.15am flight ...
8,https://www.tripadvisor.com/Airline_Review-d87...,Air China,"6,004 reviews",6004.0,China,[3.0],"[Great trip, decent food, great cabin crew, ME...",[The cabin crew was very sweet and caring. I f...
9,https://www.tripadvisor.com/Airline_Review-d87...,South African Airways,"4,883 reviews",4883.0,Africa,[3.5],"[Happy Customer’s, Avoid at all costs - zero c...",[I started writing this while in the plane. Th...


After the reviews are extracted, the Rating, Review_Title, and Review_Text lists are appended to the airline dataframe. The reviews are saved as a json file titled "airline_reviews.json".

In [ ]:
# Split reviews on | and return the last element.
airlines_df["airline_"] = airlines_df["airline_link"].apply(lambda x: x.split("|", 1)[-1] if "| " in x else x)

*Plot Total Review Distributions*

In [ ]:
# Seaborn plot

color = sns.color_palette()

fig = px.histogram(airlines_df, x="review_count",  width=800, height=400)
fig.update_traces(marker_color="pink",marker_line_color='orchid',
                  marker_line_width=1.5)
fig.update_layout(title_text='Airline review total distribution')
fig.show()

### Descriptive Statistics

In [ ]:
punctuation = set(punctuation)      # punctuation
tw_punct = punctuation - {"#"}

sw = stopwords.words("english")     # stopwords and null removal
sw = sw + ['nan']

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# Function for descriptive statistics
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Returns a list of descriptive statistics
    """

    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = num_unique_tokens/num_tokens
    num_characters = sum(len(token) for token in tokens)
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])    

# Function to remove stop words

def remove_stop(tokens) :
    return[t for t in tokens if t not in sw]

# Function to remove punctuation
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

# Function to tokenize text

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    return([item.lower() for item in whitespace_pattern.split(text)])
    
# Function to prepare pipline with necessary functions

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

# Construct pipeline using methods above

full_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]


The functions above handle a variety of data preparation steps:

- remove_stop: removes stop words in a given text

- remove_punctuation: utilizes the string.punctuation library to remove unwanted punctuation that may be present in review texts

- tokenize: employs a whitespace pattern capable of removing a variety of white space elements in a text.

- prepare: applies relevant data processing and cleaning steps to a given text using a designated pipeline

The full_pipline variable aggregates all of the necessary data handling steps. It is used with the apply() method below to create "title_tokens" and "summary_tokens".

In [ ]:
# Create tokens for review titles and summaries

airlines_df['title_tokens'] = airlines_df['Review_Title'].apply(prepare, pipeline=full_pipeline)
airlines_df['summary_tokens'] = airlines_df['Review_Text'].apply(prepare, pipeline=full_pipeline)

In [30]:
# Call descriptive statistics functions

for i in range(10):
    print(airlines_df["airline_name"][i])
    print(descriptive_stats(airlines_df["title_tokens"][i]))
    print("\n")


Emirates
There are 365 tokens in the data.
There are 201 unique tokens in the data.
There are 2462 characters in the data.
The lexical diversity is 0.551 in the data.
[365, 201, 0.5506849315068493, 2462]


LATAM Airlines
There are 370 tokens in the data.
There are 212 unique tokens in the data.
There are 2385 characters in the data.
The lexical diversity is 0.573 in the data.
[370, 212, 0.572972972972973, 2385]


Lufthansa
There are 424 tokens in the data.
There are 254 unique tokens in the data.
There are 2791 characters in the data.
The lexical diversity is 0.599 in the data.
[424, 254, 0.5990566037735849, 2791]




In [ ]:
# Function for Word lemmatization

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

# Apply lemmatization

airlines_df['title_lemma'] = airlines_df['title_tokens'].apply(lambda x: word_lemmatizer(x))
airlines_df['summary_lemma'] = airlines_df['summary_tokens'].apply(lambda x: word_lemmatizer(x))

airlines_df.head()